# Изменение параметров в процессе выполнения

:::note

С этим руководством будет проще работать, если ознакомиться с разделами:

- [LangChain Expression Language (LCEL)](/docs/concepts/#langchain-expression-language)
- [Chaining runnables](/docs/how_to/sequence/)
- [Binding runtime arguments](/docs/how_to/binding/)

:::

В процессе работы вы можете захотеть проверить разные гипотезы или показать конечному пользователю различные способы использования ваших цепочек.
Для этого может понадобиться изменить какой-нибудь параметр, например температуру, или заменить одну модель на другую.

Для таких ситуаций в LCEL есть два метода, которые позволяют изменять параметры цепочки в процессе выполнения:

* `configurable_fields` — позволяет задавать определенные поля Runnable-объектов;

  Работа с этим методом связанна с использованием метода Runnable-объектов [`.bind`](/docs/how_to/binding). Но в отличие от последнего метод `configurable_fields` позволяет задавать параметры на определенном шаге работы цепочки, а не перед ее запуском.

* `configurable_alternatives` — позволяет в процессе выполнения задать список альтернативных значений параметров для определенного Runnable-объекта, которые можно менять в процессе работы цепочки.

## Изменение определенных полей

### Языковые модели

Для языковых моделей можно изменять такие параметры, как температура.

In [1]:
%pip install --upgrade --quiet gigachain gigachain-openai

import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass()

You should consider upgrading via the '/Users/jacoblee/.pyenv/versions/3.10.5/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0).configurable_fields(
    temperature=ConfigurableField(
        id="llm_temperature",
        name="LLM Temperature",
        description="The temperature of the LLM",
    )
)

model.invoke("pick a random number")

AIMessage(content='17', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 11, 'total_tokens': 12}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-ba26a0da-0a69-4533-ab7f-21178a73d303-0')

В примере выше параметр `temperature` задан с промощью [`ConfigurableField`](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.utils.ConfigurableField.html#langchain_core.runnables.utils.ConfigurableField), что позволяет менять его в процессе работы.
Используем для этого метод [`with_config`](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable.with_config):

In [3]:
model.with_config(configurable={"llm_temperature": 0.9}).invoke("pick a random number")

AIMessage(content='12', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 11, 'total_tokens': 12}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-ba8422ad-be77-4cb1-ac45-ad0aae74e3d9-0')

Обратите внимание, что переданный параметр `llm_temperature` совпадает со значением поля `id` заданного при вызове `ConfigurableField`.

Изменять параметры можно и в рамках цепочки.

In [4]:
prompt = PromptTemplate.from_template("Pick a random number above {x}")
chain = prompt | model

chain.invoke({"x": 0})

AIMessage(content='27', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 14, 'total_tokens': 15}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-ecd4cadd-1b72-4f92-b9a0-15e08091f537-0')

In [5]:
chain.with_config(configurable={"llm_temperature": 0.9}).invoke({"x": 0})

AIMessage(content='35', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 14, 'total_tokens': 15}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-a916602b-3460-46d3-a4a8-7c926ec747c0-0')

### Экземпляры HubRunnables

Экземпляры HubRunnables используются, если нужно заменить промпт.

In [6]:
from langchain.runnables.hub import HubRunnable

prompt = HubRunnable("rlm/rag-prompt").configurable_fields(
    owner_repo_commit=ConfigurableField(
        id="hub_commit",
        name="Hub Commit",
        description="The Hub commit to pull from",
    )
)

prompt.invoke({"question": "foo", "context": "bar"})

ChatPromptValue(messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: foo \nContext: bar \nAnswer:")])

In [7]:
prompt.with_config(configurable={"hub_commit": "rlm/rag-prompt-llama"}).invoke(
    {"question": "foo", "context": "bar"}
)

ChatPromptValue(messages=[HumanMessage(content="[INST]<<SYS>> You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.<</SYS>> \nQuestion: foo \nContext: bar \nAnswer: [/INST]")])

## Настройка альтернативных значений



### Для языковых моделей

С помощью метода `configurable_alternatives()` вы можете менять шаги выполнения цепочки на заданные альтернативные значения. 
Пример демонстрирует замену языковых моделей.

In [8]:
%pip install --upgrade --quiet langchain-anthropic

import os
from getpass import getpass

os.environ["ANTHROPIC_API_KEY"] = getpass()

You should consider upgrading via the '/Users/jacoblee/.pyenv/versions/3.10.5/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [18]:
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(temperature=0).configurable_alternatives(
    # Задает id поля.
    # id можно использовать для изменения поля при созданиии итогового экземпляра Runnable
    ConfigurableField(id="llm"),
    # Поле `default_key` содержит информцию о модели, которая используется по умолчанию (ChatAnthropic)
    default_key="anthropic",
    # Поле `openai` задает возможность использовать `ChatOpenAI()`
    openai=ChatOpenAI(),
    # Поле `gpt4` задает возможность использовать `ChatOpenAI(model="gpt-4")`
    gpt4=ChatOpenAI(model="gpt-4"),
    # Вы можете добавить другие варианты.
)
prompt = PromptTemplate.from_template("Tell me a joke about {topic}")
chain = prompt | llm

AIMessage(content="Here's a bear joke for you:\n\nWhy don't bears wear socks? \nBecause they have bear feet!\n\nHow's that? I tried to come up with a simple, silly pun-based joke about bears. Puns and wordplay are a common way to create humorous bear jokes. Let me know if you'd like to hear another one!", response_metadata={'id': 'msg_018edUHh5fUbWdiimhrC3dZD', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 13, 'output_tokens': 80}}, id='run-775bc58c-28d7-4e6b-a268-48fa6661f02f-0')

In [19]:
# По умолчанию используется модель Anthropic
chain.invoke({"topic": "bears"})

AIMessage(content=" Here's a silly joke about bears:\n\nWhat do you call a bear with no teeth?\nA gummy bear!")

In [19]:
# Используйте `.with_config(configurable={"llm": "openai"})`, чтобы задать модель
chain.with_config(configurable={"llm": "openai"}).invoke({"topic": "bears"})

AIMessage(content="Why don't bears like fast food?\n\nBecause they can't catch it!", response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 13, 'total_tokens': 28}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-7bdaa992-19c9-4f0d-9a0c-1f326bc992d4-0')

In [20]:
# Или передайте значение поля `default_key` чтобы импользовать модель, заданную по умолчанию
chain.with_config(configurable={"llm": "anthropic"}).invoke({"topic": "bears"})

AIMessage(content="Here's a bear joke for you:\n\nWhy don't bears wear socks? \nBecause they have bear feet!\n\nHow's that? I tried to come up with a simple, silly pun-based joke about bears. Puns and wordplay are a common way to create humorous bear jokes. Let me know if you'd like to hear another one!", response_metadata={'id': 'msg_01BZvbmnEPGBtcxRWETCHkct', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 13, 'output_tokens': 80}}, id='run-59b6ee44-a1cd-41b8-a026-28ee67cdd718-0')

### Для промптов

Вы также можете задавать альтернативные промпты.


In [22]:
llm = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0)
prompt = PromptTemplate.from_template(
    "Tell me a joke about {topic}"
).configurable_alternatives(
    # Задает id поля
    # id можно использовать для изменения поля при созданиии итогового экземпляра Runnable
    ConfigurableField(id="prompt"),
    # Поле `default_key` указывает, что по умолчанию будет генерироваться шутка.
    default_key="joke",
    # Поле `poem` задает возможность сгенерироват стих.
    poem=PromptTemplate.from_template("Write a short poem about {topic}"),
    # Вы можете добавить другие варианты.
)
chain = prompt | llm

AIMessage(content="Here's a bear joke for you:\n\nWhy don't bears wear socks? \nBecause they have bear feet!", response_metadata={'id': 'msg_01DtM1cssjNFZYgeS3gMZ49H', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 13, 'output_tokens': 28}}, id='run-8199af7d-ea31-443d-b064-483693f2e0a1-0')

In [26]:
# По умолчанию генерируется шутка
chain.invoke({"topic": "bears"})

AIMessage(content=" Here's a silly joke about bears:\n\nWhat do you call a bear with no teeth?\nA gummy bear!")

In [23]:
# Передайте значение `poem`, чтобы сгенерировать стих
chain.with_config(configurable={"prompt": "poem"}).invoke({"topic": "bears"})

AIMessage(content="Here is a short poem about bears:\n\nMajestic bears, strong and true,\nRoaming the forests, wild and free.\nPowerful paws, fur soft and brown,\nCommanding respect, nature's crown.\n\nForaging for berries, fishing streams,\nProtecting their young, fierce and keen.\nMighty bears, a sight to behold,\nGuardians of the wilderness, untold.\n\nIn the wild they reign supreme,\nEmbodying nature's grand theme.\nBears, a symbol of strength and grace,\nCaptivating all who see their face.", response_metadata={'id': 'msg_01Wck3qPxrjURtutvtodaJFn', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 13, 'output_tokens': 134}}, id='run-69414a1e-51d7-4bec-a307-b34b7d61025e-0')

### Для промптов и моделей

Параметры промптов и моделей можно задавать одновременно.

In [25]:
llm = ChatAnthropic(temperature=0).configurable_alternatives(
    # Задает id поля.
    # id можно использовать для изменения поля при созданиии итогового экземпляра Runnable
    ConfigurableField(id="llm"),
    # Поле `default_key` содержит информцию о модели, которая используется по умолчанию (ChatAnthropic)
    default_key="anthropic",
    # Поле `openai` задает возможность использовать `ChatOpenAI()`
    openai=ChatOpenAI(),
    # Поле `gpt4` задает возможность использовать `ChatOpenAI(model="gpt-4")`
    gpt4=ChatOpenAI(model="gpt-4"),
    # Вы можете добавить другие варианты.
)
prompt = PromptTemplate.from_template(
    "Tell me a joke about {topic}"
).configurable_alternatives(
    # Задает id поля
    # id можно использовать для изменения поля при созданиии итогового экземпляра Runnable
    ConfigurableField(id="prompt"),
    # Поле `default_key` указывает, что по умолчанию будет генерироваться шутка.
    default_key="joke",
    # Поле `poem` задает возможность сгенерироват стих.
    poem=PromptTemplate.from_template("Write a short poem about {topic}"),
    # Вы можете добавить другие варианты.
)
chain = prompt | llm

AIMessage(content="In the forest deep and wide,\nBears roam with grace and pride.\nWith fur as dark as night,\nThey rule the land with all their might.\n\nIn winter's chill, they hibernate,\nIn spring they emerge, hungry and great.\nWith claws sharp and eyes so keen,\nThey hunt for food, fierce and lean.\n\nBut beneath their tough exterior,\nLies a gentle heart, warm and superior.\nThey love their cubs with all their might,\nProtecting them through day and night.\n\nSo let us admire these majestic creatures,\nIn awe of their strength and features.\nFor in the wild, they reign supreme,\nThe mighty bears, a timeless dream.", response_metadata={'token_usage': {'completion_tokens': 133, 'prompt_tokens': 13, 'total_tokens': 146}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-5eec0b96-d580-49fd-ac4e-e32a0803b49b-0')

In [29]:
# Итоговая цепочка вернет стих, сгенерированный с помощью OpenAI
chain.with_config(configurable={"prompt": "poem", "llm": "openai"}).invoke(
    {"topic": "bears"}
)

AIMessage(content="In the forest, where tall trees sway,\nA creature roams, both fierce and gray.\nWith mighty paws and piercing eyes,\nThe bear, a symbol of strength, defies.\n\nThrough snow-kissed mountains, it does roam,\nA guardian of its woodland home.\nWith fur so thick, a shield of might,\nIt braves the coldest winter night.\n\nA gentle giant, yet wild and free,\nThe bear commands respect, you see.\nWith every step, it leaves a trace,\nOf untamed power and ancient grace.\n\nFrom honeyed feast to salmon's leap,\nIt takes its place, in nature's keep.\nA symbol of untamed delight,\nThe bear, a wonder, day and night.\n\nSo let us honor this noble beast,\nIn forests where its soul finds peace.\nFor in its presence, we come to know,\nThe untamed spirit that in us also flows.")

In [26]:
# We can always just configure only one if we want
chain.with_config(configurable={"llm": "openai"}).invoke({"topic": "bears"})

AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!", response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 13, 'total_tokens': 26}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-c1b14c9c-4988-49b8-9363-15bfd479973a-0')

### Сохранение конфигураций

Настроенные цепочки можно сохранять как отдельные объекты.

In [27]:
openai_joke = chain.with_config(configurable={"llm": "openai"})

openai_joke.invoke({"topic": "bears"})

AIMessage(content="Why did the bear break up with his girlfriend? \nBecause he couldn't bear the relationship anymore!", response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 13, 'total_tokens': 33}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-391ebd55-9137-458b-9a11-97acaff6a892-0')

## Смотрите также

- [Привязка аргументов](/docs/how_to/binding)